Forked from excellent kernel : https://www.kaggle.com/jsaguiar/updated-0-792-lb-lightgbm-with-simple-features

From Kaggler : https://www.kaggle.com/jsaguiar

In [1]:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Helper Functions
ToDo: Try doing a different type of encoding for the categorical variables

In [2]:
def one_hot_encoder(df, nan_as_category = True):
    # get original columns
    original_columns = list(df.columns)
    # get all of the categorical columns
    categorical_cols = [col for col in df.columns if df[col].dtype == "object"]
    # use pandas pd.get_dummies to turn the categorical columns into dummy variables
    df = pd.get_dummies(df, dumm_na = nan_as_category)
    # return the dataframe and the list of new columns
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

# processing the main training set

# Processing Bureau and bureau_balance

# Processing previous applications

# Processing POS CASH balance

# Processing Installments

# Processing credit_card_balance

# ML algs